In [1]:
import torch
import numpy as np
from kaolin.render.camera import Camera, \
    generate_rays, generate_pinhole_rays, \
    generate_centered_pixel_coords, generate_centered_custom_resolution_pixel_coords

In [2]:
camera = Camera.from_args(
    eye=torch.tensor([4.0, 4.0, 4.0]),
    at=torch.tensor([0.0, 0.0, 0.0]),
    up=torch.tensor([0.0, 1.0, 0.0]),
    fov=30 * np.pi / 180,  # In radians
    x0=0.0, y0=0.0,
    width=800, height=800,
    near=1e-2, far=1e2,
    dtype=torch.float64,
    device='cuda'
)
print(camera)

CameraExtrinsics of 1 cameras, of coordinate system: 
tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]], device='cuda:0', dtype=torch.float64). 
Camera #0 View Matrix: 
tensor([[[ 0.7071,  0.0000, -0.7071,  0.0000],
         [-0.4082,  0.8165, -0.4082,  0.0000],
         [ 0.5774,  0.5774,  0.5774, -6.9282],
         [ 0.0000,  0.0000,  0.0000,  1.0000]]], device='cuda:0',
       dtype=torch.float64),
Camera #0 Inverse View Matrix: 
tensor([[[ 0.7071, -0.4082,  0.5774,  4.0000],
         [ 0.0000,  0.8165,  0.5774,  4.0000],
         [-0.7071, -0.4082,  0.5774,  4.0000],
         [ 0.0000,  0.0000,  0.0000,  1.0000]]], device='cuda:0',
       dtype=torch.float64)

PinholeIntrinsics of 1 cameras of resolution 800x800.
Camera #0: {'x0': 0.0, 'y0': 0.0, 'focal_x': 1492.820323027551, 'focal_y': 1492.820323027551}



In [3]:
# General raygen functiontional version -- will invoke raygen according to the camera lens type
ray_orig, ray_dir = generate_rays(camera)
print(f'Created a ray grid of dimensions: {ray_orig.shape}')
print('Ray origins:')
print(ray_orig)
print('Ray directions:')
print(ray_dir)
print('\n')

Created a ray grid of dimensions: torch.Size([640000, 3])
Ray origins:
tensor([[4.0000, 4.0000, 4.0000],
        [4.0000, 4.0000, 4.0000],
        [4.0000, 4.0000, 4.0000],
        ...,
        [4.0000, 4.0000, 4.0000],
        [4.0000, 4.0000, 4.0000],
        [4.0000, 4.0000, 4.0000]], device='cuda:0', dtype=torch.float64)
Ray directions:
tensor([[-0.8191, -0.3356, -0.4652],
        [-0.8188, -0.3357, -0.4657],
        [-0.8185, -0.3357, -0.4662],
        ...,
        [-0.2618, -0.7446, -0.6141],
        [-0.2613, -0.7444, -0.6144],
        [-0.2608, -0.7443, -0.6148]], device='cuda:0', dtype=torch.float64)




/home/quantm/anaconda3/envs/kaolin/lib/python3.11/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1720538456841/work/aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [4]:
# General raygen function OOP version -- can also be invoked directly on the camera object
ray_org, ray_dir = camera.generate_rays()
print(f'Created a ray grid of dimensions: {ray_org.shape}')
print('Ray origins:')
print(ray_org)
print('Ray directions:')
print(ray_dir)
print('\n')

Created a ray grid of dimensions: torch.Size([640000, 3])
Ray origins:
tensor([[4.0000, 4.0000, 4.0000],
        [4.0000, 4.0000, 4.0000],
        [4.0000, 4.0000, 4.0000],
        ...,
        [4.0000, 4.0000, 4.0000],
        [4.0000, 4.0000, 4.0000],
        [4.0000, 4.0000, 4.0000]], device='cuda:0', dtype=torch.float64)
Ray directions:
tensor([[-0.8191, -0.3356, -0.4652],
        [-0.8188, -0.3357, -0.4657],
        [-0.8185, -0.3357, -0.4662],
        ...,
        [-0.2618, -0.7446, -0.6141],
        [-0.2613, -0.7444, -0.6144],
        [-0.2608, -0.7443, -0.6148]], device='cuda:0', dtype=torch.float64)




In [5]:
# A specific raygen function can also be invoked directly. You may also add your own custom raygen functions that way
ray_org, ray_dir = generate_pinhole_rays(camera)
print(f'Created a ray grid of dimensions: {ray_org.shape}')
print('Ray origins:')
print(ray_org)
print('Ray directions:')
print(ray_dir)
print('\n')

Created a ray grid of dimensions: torch.Size([640000, 3])
Ray origins:
tensor([[4.0000, 4.0000, 4.0000],
        [4.0000, 4.0000, 4.0000],
        [4.0000, 4.0000, 4.0000],
        ...,
        [4.0000, 4.0000, 4.0000],
        [4.0000, 4.0000, 4.0000],
        [4.0000, 4.0000, 4.0000]], device='cuda:0', dtype=torch.float64)
Ray directions:
tensor([[-0.8191, -0.3356, -0.4652],
        [-0.8188, -0.3357, -0.4657],
        [-0.8185, -0.3357, -0.4662],
        ...,
        [-0.2618, -0.7446, -0.6141],
        [-0.2613, -0.7444, -0.6144],
        [-0.2608, -0.7443, -0.6148]], device='cuda:0', dtype=torch.float64)




In [6]:
# By using a custom grid input, other effects like lower resolution images can be supported
height = 200
width = 400
pixel_grid = generate_centered_custom_resolution_pixel_coords(camera.width, camera.height, width, height, camera.device)
ray_org, ray_dir = generate_pinhole_rays(camera, pixel_grid)
print(f'Created a ray grid of different dimensions from camera image plane resolution: {ray_org.shape}')
print('Ray origins:')
print(ray_org)
print('Ray directions:')
print(ray_dir)
print('\n')

Created a ray grid of different dimensions from camera image plane resolution: torch.Size([80000, 3])
Ray origins:
tensor([[4.0000, 4.0000, 4.0000],
        [4.0000, 4.0000, 4.0000],
        [4.0000, 4.0000, 4.0000],
        ...,
        [4.0000, 4.0000, 4.0000],
        [4.0000, 4.0000, 4.0000],
        [4.0000, 4.0000, 4.0000]], device='cuda:0', dtype=torch.float64)
Ray directions:
tensor([[-0.8188, -0.3365, -0.4652],
        [-0.8182, -0.3366, -0.4662],
        [-0.8175, -0.3367, -0.4672],
        ...,
        [-0.2634, -0.7443, -0.6137],
        [-0.2625, -0.7440, -0.6144],
        [-0.2615, -0.7438, -0.6151]], device='cuda:0', dtype=torch.float64)


